<a href="https://colab.research.google.com/github/M-Taha-98/Sales-and-Profitability-Analysis/blob/main/Superstore_Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Superstore Sales Profitability Analysis**

##Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re

## Read/Explore Dataset

In [ ]:
# Read the raw dataset

raw_data = pd.read_csv('Superstore Dataset_raw.csv', encoding='latin1')

In [ ]:
raw_data.head()

In [ ]:
raw_data.shape  #9994 rows and 21 columns in raw dataset

In [ ]:
raw_data.info()

# The data type for Order Date and Dhip Date needs to be changed to datetime

In [ ]:
raw_data.isnull().sum()  #no missing values

In [ ]:
#Statistical Summary

raw_data.describe()

## Data Pre-processing

####Drop extra columns

In [ ]:
# Creating a copy of dataframe to use in analysis

df = raw_data.copy()

In [ ]:
# Dropping columns redundant column not useful for analysis

df.drop(columns = ["Country"], inplace = True)  #drop Country column

In [ ]:
df.shape

####Data type correction

In [ ]:
# Correcting datatype from string to datetime

df['Order Date'] = pd.to_datetime(df['Order Date'], format = '%m/%d/%Y')

In [ ]:
# Correcting datatype from string to datetime

df['Ship Date'] = pd.to_datetime(df['Ship Date'], format = '%m/%d/%Y')

In [ ]:
df.info()

####Data Validation

In [ ]:
# Validating that all Ship Dates follow or are equal to Order Date and not the other way around

len(df[df['Ship Date'] > df['Order Date']])

In [ ]:
len(df[df['Ship Date'] == df['Order Date']])  #543-519 = 24 order IDs have been delayed (were same day shipping but not shipped the same day)

In [ ]:
df['Ship Mode'].value_counts()

####Statistical Summary

In [ ]:
# Including only numeric features

df.describe(include= ['int64','float64'])

**observations:**
  - majority (75% of data) is on the lower end of pricing and hence the generated profit.
  - 75% of data lies below sale price of approx  &#36;209 while max sale price is &#36;22,600 which indicates presence of outliers.

  - majority (75%) of items bought from this store are discounted at 20%, few items do have larger discounts such as 80%.

  - superstore has incurred loss (negative profit) on some orders.

In [ ]:
# Including only non-numeric features

df.describe(include= ['object'])

- There is no errors in customer ID and customer name columns, each customer is represented by unique customer ID
- The most common shipping method is Standard Class
- Wlliam Brown has placed most orders = 37
- A customer can place more than 1 order on same/different day hence there will be duplicate rows which is not an issue in the dataset
- The most ordered item is Staple envelope
- Consumer segment has placed the most number of orders, but that doesnot necessarily mean it brings in most sales.
- Similarly most orders are for Office Supplies, but that doesnot necessarily mean it brings in most sales.
- Most orders have been shipped to California, but is that the state bringing in most revenue?

####Duplicate Values

In [ ]:
# Checking for duplicates in the data

df.duplicated().sum()  #no duplicates

In [ ]:
df.duplicated(subset=["Customer ID"]).sum()  #since a customer can place ore than 1 order, duplicates are not an issue

## Data Transformation

####Data Cleaning

**Remove leading and trailing spaces:**

In [ ]:
# Define function to remove leading and trailing characters

for col in df.select_dtypes(include='object'):
    df[col] = df[col].str.strip()

**Remove any leading or trailing unwanted characters from customer name:**

In [ ]:
df[df['Customer Name'].str.contains('Jason', case=False, na=False)].sample(4)

<font size="3">Jason Fortune- is an error since there is an unwanted trailing character which needs to be cleaned</font>

In [ ]:
# Define custom function to remove unwanted characters from Customer Name field

def clean_edges(text):
    if isinstance(text, str):  # check if input is a string
        return re.sub(r"^[^a-zA-Z]+|[^a-zA-Z]+$", "", text) # remove anything except alphabets from start and end
    else:
        return text  # if it's not a string (maybe NaN), return as it is

In [ ]:
df['Customer Name'] = df['Customer Name'].apply(clean_edges)

**Capitalizing the Customer ID and Order ID:**

In [ ]:
df['Customer ID'] = df['Customer ID'].str.upper()
df['Order ID'] = df['Order ID'].str.upper()

**Replace empty string with Null in object columns:**

In [ ]:
# Define function to replace empty string with Null

def clean_nulls(text):
    if isinstance(text, str):   # check if input is a string
        if text.lower() == 'n/a':
            return None
        elif text == '':
            return np.nan
        else:
            return text
    else:
        return text

In [ ]:
for col in df.select_dtypes(include='object'):
    df[col] = df[col].apply(clean_nulls)

In [ ]:
print(f"The order date ranges between {df['Order Date'].dt.year.min()} and {df['Order Date'].dt.year.max()}")
print(f"The ship date ranges between {df['Ship Date'].dt.year.min()} and {df['Ship Date'].dt.year.max()}")

####Adding Custom Columns

In [ ]:
# Adding Cost Column

df['Cost'] = df['Sales'] - df['Profit']

In [ ]:
# Adding Delay Column

df['Delay'] = df['Ship Date'] - df['Order Date']

In [ ]:
# Adding Gross Sales column

df['PreDiscountSales'] = df['Sales'] / (1 - df['Discount'])

In [ ]:
#df['Profit Margin'] = df['Profit'] / df['Sales']

In [ ]:
# Overall Profit by Date of Order

b = df.groupby('Order Date')['Profit'].sum()

In [ ]:
plt.plot(b.index, b.values)

plt.xlabel('Order Date')
plt.ylabel('Profit ($)')
plt.title('Profit Over Time (Daily Trend)')
plt.xticks(rotation=45)

plt.show()

## Non-Graphical Analysis

In [ ]:
categorical  = df.select_dtypes(include = "object")

In [ ]:
for i in categorical:
  print(f"{i} has {df[i].nunique()} values")
  print(f"{df[i].unique()}\n")

In [ ]:
for i in categorical:
  display(df[i].value_counts(normalize = True)*100)

## Bivariate Analysis

###### **Sales vs Category / Sub-Category:** Compare Sales across product categories and subcategories

In [ ]:
# Sales across Product Category per order basis

order_category_sales = df.groupby(['Order ID', 'Category'])['Sales'].sum().reset_index() #total sales for each category within an order.

In [ ]:
# Plotting the Barplot
ax = sns.barplot(order_category_sales, x = 'Category', y = 'Sales', estimator='median', palette = 'coolwarm')

# Annotate data labels
for p in ax.patches:
    ax.annotate(
        format(p.get_height(), '.1f'),  # format height to 1 decimal
        (p.get_x() + p.get_width() / 2., p.get_height()),  # position at center of bar
        ha = 'center',  # horizontal alignment
        va = 'bottom',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(0, 5),  # small shift up
        textcoords='offset points'
    )

# Setting title
plt.title("Median Sales Per Order Per Product Category", fontsize = 10)

# Display the plot
plt.show()

#Technology products make the highest median sales: $219

In [ ]:
# Mean and Median Sales across Categories per order basis

order_category_sales.groupby('Category')['Sales'].agg(['mean', 'median'])  #sales of office-supplies items is most skewed.

# All category sales are right-skewed
# Furniture: mean is 2x median
# Office Supplies: mean is 4x median
# Technology: mean is 2.5x median

In [ ]:
# Sales across Product Sub-Category per order basis

order_sub_category_sales = df.groupby(['Order ID', 'Sub-Category'])['Sales'].sum().reset_index() #total sales for each sub-category within an order.

In [ ]:
# Set the canvas size
plt.figure(figsize =(11,6))

# Plotting the Barplot
ax = sns.barplot(order_sub_category_sales, x = 'Sub-Category', y = 'Sales', estimator='median', palette = 'Set2',errorbar=None )

# Annotate data labels
for p in ax.patches:
    ax.annotate(
        format(p.get_height(), '.1f'),  # format height to 1 decimal
        (p.get_x() + p.get_width() / 2., p.get_height()),  # position at center of bar
        ha = 'center',  # horizontal alignment
        va = 'bottom',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(0, 3),  # small shift up
        textcoords='offset points'
    )

# Plot customization
plt.xticks(rotation=45)
plt.title("Median Sales Per Order Per Product Sub-Category", fontsize = 12)

# Display the plot
plt.show()

#Copiers make the highest average sales: $1100, this is not a outlier because copiers are expensive item compared to rest of product catalogue.

In [ ]:
df_unique = df.groupby('Order ID').first().reset_index()  # equivalent of distinct count

In [ ]:
plt.figure(figsize =(7,2))

ax  = sns.countplot(df[df['Sub-Category']=='Copiers'],y = 'Segment', width=0.5,  palette = 'Set2')

for p in ax.patches:
    ax.annotate(
        format(p.get_width(), '.0f'),  # format height to 1 decimal
        (p.get_x() + p.get_width(), p.get_y() + p.get_height() / 2.),  # position at center of bar
        ha = 'left',  # horizontal alignment
        va = 'center',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(4, 0),  # small shift up
        textcoords='offset points'
    )
plt.xlabel("Total Copiers Ordered")

plt.show()

<font size="3">

- Copier (sub-category bringing in most sales on average) is being most ordered by consumers. </font>

###### **Profit vs Category / Sub-Category:** Compare Profit across product categories and subcategories

In [ ]:
# Profit across Product Category per order basis

order_category_profit = df.groupby(['Order ID', 'Category'])['Profit'].sum().reset_index() #total profit for each category within an order.

In [ ]:
#Barplot

ax = sns.barplot(order_category_profit, x = 'Category', y = 'Profit', estimator='median')

for p in ax.patches:
    ax.annotate(
        format(p.get_height(), '.1f'),  # format height to 1 decimal
        (p.get_x() + p.get_width() / 2., p.get_height()),  # position at center of bar
        ha = 'center',  # horizontal alignment
        va = 'bottom',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(0, 5),  # small shift up
        textcoords='offset points'
    )
plt.title("Median Profit Per Order Per Product Category")

plt.show()

#Tech category is most profitable (3x than Office Supplies category) on average on per order basis.

In [ ]:
# Mean and Median Profit across Categories per order basis

order_category_profit.groupby('Category')['Profit'].agg(['mean', 'median'])

###### **Profit Margin Per Category / Sub-Category**

In [ ]:
# Total Sales and Profit across Product Category per order basis

order_category = df.groupby(['Order ID', 'Category']).agg(Total_Sales=('Sales', 'sum'),Total_Profit=('Profit', 'sum')).reset_index()

In [ ]:
# Profit Margin across Product Category per order basis

order_category['Profit Margin'] = (order_category['Total_Profit'] / order_category['Total_Sales']) * 100

In [ ]:
# Mean and Median Profit Margin across Categories per order basis

order_category.groupby('Category')['Profit Margin'].agg(['mean', 'median'])

**Comparision:**

In [ ]:
# Sales, Profit and Profit Margin across Product Category comparision on per order basis

result_table  = pd.concat([
            order_category_sales.groupby('Category')['Sales'].agg(['median']).T.rename(index={'median': 'Sales'})
          , order_category_profit.groupby('Category')['Profit'].agg(['median']).T.rename(index={'median': 'Profit'})
          , order_category.groupby('Category')['Profit Margin'].agg(['median']).T.rename(index={'median': 'Profit Margin'})
           ], axis=0)

In [ ]:
result_table

Office Supplies has least average sales per order but has highest profit margin, while furniture has lowest profit margin despite roughly equal average sales as technology items per order, hence increase inventory of office supplies items but re-consider pricing or discount of furniture items.

###### **Profit vs Sales:** Identify if higher sales always lead to higher profits or if losses exist in high sales


In [ ]:
# Unique records in the dataset

df['Order ID'].nunique()

In [ ]:
order_total_category = df.groupby(['Order ID','Category']).agg(Total_Sales=('Sales', 'sum'),Total_Profit=('Profit', 'sum')).reset_index()

In [ ]:
# Joint plot of Sales vs Profit across Product Category
g = sns.jointplot(data = order_total_category, x= 'Total_Sales', y='Total_Profit', kind='scatter', hue = 'Category', s=70, alpha=0.7,
    height=8 ,ratio=5)

# Customizing Plot
g.fig.suptitle('Sales ($) vs Profit ($)', fontsize=12, y=1)
g.ax_joint.legend(title='Category', bbox_to_anchor = (1.5,1), title_fontsize='10', fontsize='9',frameon=True)
g.fig.set_size_inches(8,6)

# Display plot
plt.tight_layout()
plt.show()

<font size="3">It is observed that:
-  High sales do lead to higher profits (between &#36;4000 and &#36;8000) especially for certain technology products.
- However there are roughly equal number of high sale orders of tech products on which the company has incurred losses in range of &#36;2000 to &#36;4000.
- Office Supply items have the highest negative profit margin on per order basis (evident by steepest slope).
</font>

In [ ]:
# Total Sales and Profit across Product Category and Sub-Category per order basis

order_total_subcategory = df.groupby(['Order ID','Category','Sub-Category']).agg(Total_Sales=('Sales', 'sum'),Total_Profit=('Profit', 'sum')).reset_index()

In [ ]:
# Joint plot of Sales vs Profit for Technology Products

g = sns.jointplot(data = order_total_subcategory[order_total_subcategory['Category']=='Technology'], x= 'Total_Sales', y='Total_Profit',
    kind='scatter', hue = 'Sub-Category',palette = 'bright' ,s=70, alpha=0.7,height=8 ,ratio=3)

# Customizing Plot
g.fig.suptitle('Sales vs Profit (Tech Products)', fontsize=16, y=1)
g.ax_joint.legend(title='Category', bbox_to_anchor = (1.6,1), title_fontsize='10', fontsize='9',frameon=True)
g.fig.set_size_inches(8,6)

# Display plot
plt.tight_layout()
plt.show()

<font size="3">It is observed that:
- High sales on per order basis come from Copiers while Machines are denting the company beacuse of significant losses in range of &#36;2000 to &#36;4000 as discussed earlier. The superstore may consider phasing out this product offering from it's portfolio.
</font>

In [ ]:
# Total Sales and Profit figures for each unique phone order

order_total_subcategory[order_total_subcategory['Sub-Category']=='Phones']

In [ ]:
temp_df = (order_total_subcategory[order_total_subcategory['Sub-Category']=='Phones']).reset_index(drop=True)

In [ ]:
# Count total rows
count_total = len(temp_df)

# Count rows where Total_Profit > 0
count_profit_positive = len(temp_df[temp_df['Total_Profit'] > 0])

# Count rows where Total_Profit < 0
count_profit_negative = len(temp_df[temp_df['Total_Profit'] < 0])

# Count rows where Total_Profit == 0
count_profit_zero = len(temp_df[temp_df['Total_Profit'] == 0])

# Display results
print(f"Phone orders:\n\n{(count_profit_positive/count_total)*100:.2f}% orders generated profit\n{(count_profit_negative/count_total)*100:.2f}% orders generated loss\n{(count_profit_zero/count_total)*100:.2f}% orders were break even")


<font size="3">
Phone orders (within technology category) are majorly profitable hence the superstore can safely continue their sales.
</font>

######  **Profit vs Sales:** Analyze which customer segments contribute the most to profit

In [ ]:
# Total Sales and Profit across Regions and Customer Segmnt on per order basis

order_region_segment = df.groupby(['Order ID', 'Region', 'Segment']).agg(Total_Sales=('Sales', 'sum'),Total_Profit=('Profit', 'sum')).reset_index()

In [ ]:
# Adding Profit Margin column

order_region_segment['Profit Margin'] = (order_region_segment['Total_Profit'] / order_region_segment['Total_Sales']) * 100

In [ ]:
# Segment Wise breakdown of Profit

order_region_segment.groupby('Segment')['Total_Profit'].agg(['mean', 'median', 'sum', 'count'])

<font size="3">

 - Consumers bring in the most profit and is the largest demographic as well.
 - On per order basis, highest median profit comes from corporate orders which is expected under the assumption that they either buy bulk quantity or pricier items such as technology products.
</font>

In [ ]:
# Set Canvas Size
fig = plt.figure(figsize=(12, 6))

# Main Plot

# Add axis [left, bottom, width, height]
ax = fig.add_axes([0.2, 0.2, 0.55, 0.7])
#    fig.add_axes([left, bottom, width, height])

# Plot main graph
sns.barplot(data = order_region_segment, x = 'Segment', y = 'Total_Profit', estimator='sum',fill=False, errorbar= None,width=0.5, gap=2, linewidth=2, edgecolor = '#88bceb')

# Customize main plot
ax.set_xlabel('Segment')
ax.set_ylabel('Total Profit')
ax.set_title("Total Profit By Segment", fontdict={'fontsize':10, 'fontweight': 'normal'})

# Inset plot

# Add axis [left, bottom, width, height]
inset_ax = fig.add_axes([0.54, 0.6, 0.2, 0.25])

# Plot inset graph
sns.barplot(order_region_segment, x = 'Segment', y = 'Total_Profit', estimator='median',fill=False, errorbar= None, width=0.4, linewidth=1, edgecolor='#124f86')

# Customize inset plot
inset_ax.set_title("Median Profit By Segment",fontdict={'fontsize':8, 'fontweight': 'normal'}, y=1.02)
inset_ax.set_xlabel('')
inset_ax.set_ylabel('')
inset_ax.tick_params(axis='y', labelsize=8)
inset_ax.tick_params(axis='x', labelsize=8)

# Display plots
plt.show()

######  **Profit vs  Region / State:** Detect regional performance disparities and spot high/low-profit zones


In [ ]:
# Per-order profitability by Region (e.g., "What’s the average profit per transaction/per order in the West region?")

order_region_segment.groupby('Region')['Total_Profit'].agg(['mean', 'median','sum']).sort_values(by='sum', ascending=False)

<font size="3">
It is observed that:

 - West region outperforms others with approximately 15% more profit generated compared to East region which is the next most profitable region on per order basis.
 - Central Region is the most under-performing region with cumulative profit of approx &#36;40,000 which is roughly 3x lesser than West region.
 - Mean Profit as multiple of Median Profit by region:
   
   West: 2.8x  East: 4x  South: 3x  Central: 5x

   - This shows that there are high-value orders coming from Central region which pulls the mean order value much higher than the median: need to explore segment distribution in Central region to find if majorly corporate customers are ordering from there which are expected to make bulk quantity orders i.e. high-value orders.


</font>

In [ ]:
# Per-item profitability by Region (e.g., "What’s the median profit of individual products sold in the East region?")

pd.pivot_table(df, values = 'Profit', index = 'Region', aggfunc = ['mean','median', 'sum']).sort_values(by=('sum', 'Profit'), ascending=False)

######  **Profit Distribution by Region and Segment**

In [ ]:
# Regional Per Order/Transaction Profit Distribution by Customer Segment

pd.pivot_table(order_region_segment, values = 'Total_Profit', index = 'Segment', columns = 'Region', aggfunc = ['mean','median', 'sum']).sort_values(by=('sum', 'Central'), ascending=False)

<font size="3">
It is observed that:

 - This table confirms that in Central Region, major profit is earned from Corporate hence the large disparity between median and mean profit per order.
 - It is also noted that in East, South and West regions, consumer segment brings in the biggest share of profit per transaction while Corporate Segment is biggest profit contributor in Central region.
 - However, most profit from Corporate segment has been earned from West region, this could be merely due to more Corporate orders coming in from there due to higher Corporate concentration in the West region compared to other regions. This needs to be explored.


</font>

In [ ]:
# Order frequency across Region and Customer segment

pd.pivot_table(order_region_segment, values = 'Order ID', index = 'Region', columns = 'Segment', aggfunc = ['count'],margins=True,margins_name='Total')

<font size="3">
It is observed that:

 - Most Corporate orders come from West region confirming why most profit from Corporate segment has been earned from West region.
 - Interestingly in Central region, most orders come from Consumer segment like other regions but Consumer segment has brought in least profit per order as seen in previous table. Is that because of larger discounts in Central region for consumers compared to other regions or high costs incurred by the store in Central region which is reducing the profit margin?


</font>

In [ ]:
# Regional Per Order/Transaction Profit, Sales and Profit Margin Distribution by Customer Segment

order_region_segment.groupby(['Region','Segment']).agg({'Total_Profit':['sum'],'Total_Sales':['sum'], 'Profit Margin': ['mean', 'median']})

<font size="3">
It is observed that:

 - Unlike other regions where median profit margin (per order basis) is stable across segments, in Central region: the profit margin for Consumer segment is nearly half of Home Office segment.

 - Is that because of larger discounts in Central region for consumers compared to other regions or high costs incurred by the store in Central region can be explored.


</font>

**What percentage of orders come from each segment and region?**

In [ ]:
segment_count = order_region_segment[['Region','Segment']].value_counts().unstack()

In [ ]:
segment_count

In [ ]:
# Subplots

fig, ax = plt.subplots(figsize=(6, 6))

size = 0.3

# Calculate percentages for Segments (inner pie)

segment_percentages = (segment_count / segment_count.sum(axis=1).values.reshape(-1,1)) * 100

# Inner Pie (Segments per Region)

inner_pie = ax.pie(
    segment_count.values.flatten(),                                       # Flatten segment counts
    #labels=segment_count.columns.repeat(len(segment_count)),
    labels=[f'{p:.1f}%' for p in segment_percentages.values.flatten()],   # Show % labels
    labeldistance=0.75,
    wedgeprops=dict(width=0.25, edgecolor='white'),
    textprops={'fontsize': 8, 'color': 'black' },
    colors=['#90c1ec', '#e18d89', '#ffca8d'] * len(segment_count),
    radius=1-size,
    startangle=90,
)

# Add center legend for segments

ax.legend(
    inner_pie[0][:3],                                                     # First instance of each segment color
    segment_count.columns,
    title="Segments",
    loc='center',
    bbox_to_anchor=(1.2, 0.5)
)

# Outer Pie (Regions)

outer_pie = ax.pie(
    segment_count.sum(axis=1),
    labels = segment_count.index,
    autopct='%1.1f%%',
    pctdistance=0.85,
    startangle=90,
    wedgeprops=dict(width=0.2, edgecolor='white'),
    colors=['#FF9999', '#66B3FF', '#99FF99', '#FFCC99'],
    radius=1
)

# Equal aspect ratio ensures pie is drawn as a circle

ax.axis('equal')

# Title and Legend
'''
plt.title('Orders by Region and Segment', pad=20)
plt.legend(
    handles=outer_pie[0],
    labels = segment_count.index,
    title="Regions",
    loc="upper left",
    bbox_to_anchor=(1, 0.9)
) '''

# Display Plot

plt.tight_layout()
plt.show()

######  **Profit vs Ship Mode:** Check if certain shipping methods are more profitable or cost-intensive

In [ ]:
# Cost, Profit and Sales by Ship Mode per unique order

per_transaction = df.groupby(['Order ID','Ship Mode']).agg(Total_Sales=('Sales', 'sum'),Total_Profit=('Profit', 'sum'),Total_Cost = ('Cost', 'sum') ).reset_index()

In [ ]:
# Adding Profit Margin column

per_transaction['Profit Margin'] = (per_transaction['Total_Profit'] / per_transaction['Total_Sales']) * 100

In [ ]:
#pd.crosstab(per_transaction['Ship Mode'], per_transaction['Total_Profit'], normalize = True)

# Total Cost, Profit and Sales distribution across types of Ship Mode on per order basis

ship_mode = pd.pivot_table(per_transaction, values =['Total_Cost','Total_Profit','Total_Sales'], index = 'Ship Mode', aggfunc = ['sum'])

In [ ]:
# Adding Profit Margin column

ship_mode['Profit_Margin'] = (ship_mode[('sum', 'Total_Profit')]/ship_mode[('sum',  'Total_Sales')])*100

In [ ]:
# Sorting by Profit Margin

ship_mode.sort_values(by=('Profit_Margin'), ascending=False)

In [ ]:
# Sorting by Total Sales

ship_mode.sort_values(by=('sum',  'Total_Sales'), ascending=False)

<font size="3">
It is observed that:

 - Standard Class yields the lowest profit margin despite highest total sales and profit. This could be because it is cost-intensive (may be eating into profits due to high logistics costs).

- Track the Region with most number of standard class orders -> Track the cities within that region with most number of standard class orders -> this will help superstore narrow down specific locations where it needs to negotiate better cost with the shipping partner to increase it's profit margin.


</font>

<font size="3">
The low profit margin for Standard Class could be due to following reasons:

 1.  Standard Class handles a large volume of **low-margin orders** (e.g., bulk discounts, competitive pricing), the profit per transaction could be smaller even if total profit is high.

2. Price Sensitivity and Customer Behavior: Standard Class might attract price-sensitive customers who buy cheaper items, yielding less profit per sale/ low Average Order Value (AOV).

3. Shipping Costs: Standard Class might have lower fixed costs (e.g., no expedited logistics) but **higher variable costs per order** (e.g., longer delivery times → more handling, returns, or discounts). Due to lack of shipping cost data, this cannot be investigated.

</font>

In [ ]:
# Average Order Value (AOV) or Average Sale Per Order:

per_transaction.groupby('Ship Mode')['Total_Sales'].agg(['mean','median'])

# The median AOV for standard class is approx $147 which is nearly same as that of First Class.

In [ ]:
# Comparing across shipping methods to see if Standard Class has lower profit per transaction:

per_transaction.groupby('Ship Mode')['Total_Profit'].agg(['mean','median'])

<font size="3">

 Despite similar AOV and profit per transaction/order, Standard Class has lowest profit margin while First Class has highest profit margin.This could be due to:

 - **Price Premium for Speed**: First Class likely charges a higher shipping fee or Standard Class shipping could be free hence adding to direct profit.
 - **Fewer Discounts**: First Class orders might be less likely to use bulk discounts, preserving margin as opposed to Standard Class.
 - **Lower Operational Costs**: First Class may have lower operational cost (such as lower labor cost due to prioritized processing or fewer return/order cancellations).
 - **Customer Segment Differences**:
  - First Class buyers may purchase **higher-margin products** (even if AOV is similar).
  - Standard Class might be used for low-margin, high-volume items (e.g., essentials).


</font>

<font size="3">

 Recommended Steps:

 - **Analyze Product Mix by Shipping Method**:
 Check if Standard Class is tied to low-margin products. If yes, then bundle products to increase AOV.
 - **Promote First Class Shipping**: Since it yeilds the highest profit margin (approx 14%), hence highlight perks (faster delivery, fewer returns) to justify the price for first class shipping.
 - **Validate Hypothesis**:
  - "Standard Class’s low margin is driven by high promotional discounts."
    - Can be done by running A/B test or regression analysis with recent historical data if A/B test is not feasible. Be vary of correlation ≠ causation and cofounding variables when running regression analysis. If discounts are found to be eroding profit margins for standard shipping orders, then decision may be made to lower discounts.
 - **Analyze Cost Data**: If Standard Class orders have high logistic cost, then better terms may be negotiated with carriers such as lower bulk shipping rates.


</font>

######  **Profit vs Time (Month/Year)**: Seasonal profitability trends

In [ ]:
# Define function to label quarters

def quarter(date):
    if (date.month >= 0) and (date.month <= 3):
      return 'Q1'

    elif (date.month >= 4) and (date.month <= 6):
      return 'Q2'

    elif (date.month >= 7) and (date.month <= 9):
      return 'Q3'

    else:
      return 'Q4'

In [ ]:
# Add Quarter column

df['Quarter'] = df['Order Date'].apply(lambda x: quarter(x))

In [ ]:
# Add Year column

df['Year'] = df['Order Date'].dt.year

In [ ]:
# Add Month column

df['Month'] = df['Order Date'].dt.month

In [ ]:
# Total Seasonal Profit and Sales across each Quarter

seasonal_profit_sales = df.groupby(['Year','Quarter'])[['Profit','Sales']].sum().reset_index()

In [ ]:
# Adding Profit Margin column

seasonal_profit_sales['Profit Margin'] = (seasonal_profit_sales['Profit']/seasonal_profit_sales['Sales'])*100

In [ ]:
# Adding label for x-tick

seasonal_profit_sales['Year-Quarter'] = seasonal_profit_sales['Year'].astype(str) + '-' + seasonal_profit_sales['Quarter']

In [ ]:
# Sorting ascending by time

seasonal_profit_sales = seasonal_profit_sales.sort_values(['Year', 'Quarter'])

In [ ]:
# Subplots
fig, axes = plt.subplots(nrows=3,ncols =1 ,figsize=(10,13))

#first plot: Profit
axes[0].plot(seasonal_profit_sales['Year-Quarter'], seasonal_profit_sales['Profit'], marker='o', linestyle='--', color= '#5b5f9a')
axes[0].set_ylabel('Profit ($)')
axes[0].set_title("Quarterly Trends (2014-2017)", fontsize=11)
axes[0].tick_params(axis='x', labelrotation=45)
axes[0].grid(True, linestyle='--', alpha=0.2)

#second plot: Sales
axes[1].plot(seasonal_profit_sales['Year-Quarter'], seasonal_profit_sales['Sales'], marker='o', linestyle='--', color= '#9b645e')
axes[1].set_ylabel('Sales ($)')
#axes[1].set_title("Quarterly Sales Trends (2014-2017)", fontsize=11)
axes[1].tick_params(axis='x', labelrotation=45)
axes[1].grid(True, linestyle='--', alpha=0.2)

#third plot: Profit Margin
axes[2].plot(seasonal_profit_sales['Year-Quarter'], seasonal_profit_sales['Profit Margin'], marker='o', linestyle='--', color= '#87b87e')
axes[2].set_ylabel('Profit Margin (%)')
#axes[2].set_title("Quarterly Profit Margin Trends (2014-2017)", fontsize=11)
axes[2].tick_params(axis='x', labelrotation=45)
axes[2].grid(True, linestyle='--', alpha=0.2)

# Display plot
plt.tight_layout()
plt.show()

**Quarterly Profit Trends (2014–2017)**

- Overall Growth: Profit shows a general increasing trend from 2014 to 2017, with significant spikes in late 2016 and late 2017.

- Volatility: There are frequent sharp drops after high peaks (e.g., after 2014 Q4, 2015 Q4, 2016 Q4).

- Seasonality: Q4 of each year tends to have a strong spike in profits, suggesting seasonal effects (possibly due to holiday sales).

- Recent Stability: In 2017 Q3 and Q4, profits stabilize at a relatively high level compared to earlier years.

**Quarterly Sales Trends (2014–2017)**
- Steady Growth: Sales show a strong and relatively consistent upward trend over the years.

- Sharp Drops: Noticeable sales drops occur in Q1 of each year, especially in 2015 and 2016.

- Peak in Q4: Like profits, sales peak dramatically in Q4 of each year — indicating strong end-of-year performance.

- Acceleration: The sales growth rate accelerates particularly in 2017.

**Quarterly Profit Margin Trends (2014–2017)**

- Initial Stability, Later Volatility: Profit margins are fairly stable around 12–14% from 2014 to early 2016 but become much more volatile afterward.

- High Peaks: A major peak occurs in 2017 Q1 (around 19%), suggesting exceptional efficiency/profitability for that quarter.

- Recent Decline: After peaking, profit margins decrease sharply towards the end of 2017 despite high sales.

- Decoupling: Sales and profits are growing, but margins are shrinking, indicating rising costs or inefficiencies.



**Actionable Recommendations**:
- Investigate Q4 Strategies: Leverage the success of Q4 campaigns throughout the year — analyze what drives the Q4 spikes (e.g., promotions, product launches, seasonal demand).

- Address Margin Compression: Since profit margins declined in late 2017 despite high sales, stakeholders should analyze cost structures — possibly renegotiating supplier contracts, optimizing operational costs, or increasing prices selectively.

- Smooth Out Seasonality: Explore strategies to boost Q1 sales and profits, which consistently dip — such as loyalty programs, off-season promotions, or new product introductions early in the year.

- Focus on Sustainable Growth: While sales are growing rapidly, ensuring that profit margins are maintained will be crucial. Focus should not only be on revenue growth but also on operational efficiency.

- Predict and Prepare for Volatility: Build contingency plans for quarters following strong Q4s to cushion typical profit and sales drop-offs (like in Q1).

- Deep Dive into 2017: Specially investigate why profit margins declined despite record sales — was it due to heavy discounting, rising input costs, or operational issues?

## Multivariate Analysis


######  **Profit Margin vs Time**

In [ ]:
# Define function for Weighted Discount

def WeightedDiscountRate(group):
  return (group['Discount'] * group['PreDiscountSales']).sum() / group['PreDiscountSales'].sum()

In [ ]:
# Weighted Discount Rate is a way of measuring the true economic impact of discounts by giving more importance to higher-priced items.

weighted_discounts = df.groupby('Order ID').apply(WeightedDiscountRate).reset_index(name='Weighted Discount Rate')

In [ ]:
# Merging with original dataframe to add Weighted Discount column in the original data

df_wd = df.merge(weighted_discounts, on='Order ID')

In [ ]:
# Dataframe with Unique Order IDs and product category within an order recorded within the same row

df_grouped  = df.groupby('Order ID').agg({
    'Order Date': 'first',
    'Ship Date': 'first',
    'Ship Mode': 'first',
    'Customer ID': 'first',
    'Customer Name': 'first',
    'Segment': 'first',
    'City': 'first',
    'State': 'first',
    'Postal Code': 'first',
    'Region': 'first',
    'Category': list,
    'Sales': 'sum',
    'Profit':'sum',
    'Cost':'sum',
    'Delay': 'first',
    'Quarter':'first',
    'Year':'first',
    'Month':'first',
    'PreDiscountSales':'sum'
  }).reset_index()

In [ ]:
# Adding Profit Margin column

df_grouped['Profit Margin'] = (df_grouped['Profit']/df_grouped['Sales'])*100

In [ ]:
# Adding label for x-tick

df_grouped['Year-Quarter'] = df_grouped['Year'].astype(str) + '-' + df_grouped['Quarter']

In [ ]:
# Sort values by time

df_grouped = df_grouped.sort_values(['Year', 'Quarter'])

In [ ]:
# Filtering 2017 data

df_grouped_2017 = df_grouped.query("Year== 2017").reset_index()

In [ ]:
# Dropping index column

df_grouped_2017 = df_grouped_2017.drop(columns='index', axis=0)

In [ ]:
# Canvas Size
plt.figure(figsize=(12, 6))

# Plot graph
sns.lineplot(data = df_grouped, x= df_grouped['Year-Quarter'], y= df_grouped['Profit Margin'], hue='Region',marker='o', linestyle='--', color= '#87b87e',
            style = 'Segment',errorbar = None, estimator="mean")

# Customizing the plot
plt.xlabel('')
plt.ylabel('Profit Margin (%)')
plt.title("Average Quarterly Profit Margin Trends (2014-2017)", fontsize=11)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.2)


# Show the plot
plt.legend(bbox_to_anchor=(1, 0.9))
plt.tight_layout()
plt.show()

In [ ]:
# Canvas Size
plt.figure(figsize=(12, 6))

# Plot graph
sns.lineplot(data = df_grouped_2017, x= df_grouped_2017['Quarter'], y= df_grouped_2017['Profit Margin'], hue='Region',marker='o', linestyle='--', color= '#87b87e',
            style = 'Segment',errorbar = None,  estimator="mean")

# Customizing the plot
plt.xlabel('')
plt.ylabel('Profit Margin (%)')
plt.title("Average Quarterly Profit Margin Trends (2017)", fontsize=11)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.2)


# Show the plot
plt.legend(bbox_to_anchor=(1, 0.9))
plt.tight_layout()
plt.show()

<font size="3">

- The steepest drop in average profit margin from Q3 to Q4 of 2017 is observed in Consumer segment in Central region (-3% to -17.6%), and Home Office segment in South region (19% to 3%).
- The superstore may consider further investigating the driver(s) behind this poor performance in these specific demographics:
  - were there cost spikes from manufacturers or logistic partners that eat up major chunk of profit. Example: A hurricane in the South (e.g., Hurricane Harvey in Aug 2017 in Texas and Louisiana) could have disrupted logistics, increasing freight costs.
  - were heavy discounts run in last quarter of 2017 for these segments?
  - did Q4 sales in these demographics skew towards low-margin items?


</font>

In [ ]:
# Profit Margin in each quarter of 2017 across regions and customer segments

df_grouped_2017.groupby(['Quarter', 'Region', 'Segment'])['Profit Margin'].median()

In [ ]:
df_grouped_2017['Profit Margin'].describe()

In [ ]:
# Ungrouped 2017 data

df_2017 = df.query("Year== 2017").reset_index()

In [ ]:
# Adding Gross Sales column

df_2017['PreDiscountSales'] = df_2017['Sales'] / (1 - df_2017['Discount'])

In [ ]:
# 2017 data on per order basis (including Weighted Discount column)

df_2017_weighted_discount = df_2017.groupby('Order ID').apply(
        lambda x: pd.Series({
        'Sales': x['Sales'].sum(),
        'PreDiscountSales': x['PreDiscountSales'].sum(),
        'Profit': x['Profit'].sum(),
        'WeightedDiscountRate': (x['Discount'] * x['PreDiscountSales']).sum() / x['PreDiscountSales'].sum(),
        'Region' :x['Region'].iloc[0],
        'Segment': x['Segment'].iloc[0],
        'Quarter': x['Quarter'].iloc[0]
    })
).reset_index()

In [ ]:
# Adding Profit Margin column

df_2017_weighted_discount['Profit Margin'] = (df_2017_weighted_discount['Profit']/df_2017_weighted_discount['Sales'])*100

In [ ]:
# Correlation analysis of profit margin and weighted discount rates in 2017 (all regions):

corr_matrix = df_2017_weighted_discount.corr(numeric_only = True)

corr_matrix

# high negative (-0.86) correlation exist between profit margin and weighted discount rate in 2017.

In [ ]:
# 2017 Q4 data of South and Central regions

df_2017_weighted_discount_south_central_Q4 = df_2017_weighted_discount[(df_2017_weighted_discount['Region'].isin(
    ['South','Central'])) & (df_2017_weighted_discount['Quarter'] == 'Q4')]

In [ ]:
# Correlation analysis of profit margin and weighted discount rates in 2017 (Q4), only South and Central regions:

corr_matrix_2 = df_2017_weighted_discount_south_central_Q4.corr(numeric_only = True)
corr_matrix_2

In [ ]:
# Profit Margin vs Weighted Discount Rate (South & Central Regions, Q4 2017)

# Plot Graph
g = sns.lmplot(                                 # combines regplot() and FacetGrid
    data= df_2017_weighted_discount_south_central_Q4,
    x='WeightedDiscountRate',
    y='Profit Margin',
    hue = 'Segment',
    col='Region',
    height=6,
    ci= None,
    palette = 'gist_stern'
)

# Customizing Plot
g.fig.suptitle("Profit Margin by Weighted Discount Rate (Q4, 2017)", fontsize=12)
g.fig.subplots_adjust(top=0.89)
g.set_axis_labels("Weighted Discount Rate", "Profit Margin (%)")

# Display Plot
plt.show()

<font size="3">
It can be concluded that:

- The profit margin drop in Central region has been due to high discounts (60%-80%) given to Consumer segment in the last quarter of 2017 as can be observed in the plot and high negative correlation of -0.88.
- In South region, the discounts given to Home Office segment  have ranged between 0-20% with only a couple of orders that have been discounted more than 50%. This indicates that the profit margin has been impacted by other factors such as costs/sale of low margin products.  


</font>

######  **Time (Month/Year) + Category vs Profit**: Analyze time-based trends in category-level performance

In [ ]:
# Profit and Sales across each month by Product Category

month_category = df.groupby(['Year','Month', 'Category'])[['Profit','Sales']].sum().reset_index()

In [ ]:
# Filtering data for 2016 and 2017

month_category = month_category[month_category['Year'].between(2016, 2017)].reset_index()
month_category = month_category.drop(columns = 'index', axis=0)

In [ ]:
# Label for xtick

month_category['Month-Year'] =  month_category['Month'].astype(str) + '-' + month_category['Year'].astype(str)

In [ ]:
# Sort values by time

month_category = month_category.sort_values(['Year', 'Month'])

In [ ]:
# Set Canvas size
plt.figure(figsize=(12, 6))

# Plot graph
sns.lineplot(data = month_category, x= month_category['Month-Year'], y= month_category['Profit'], hue='Category',marker='o', linestyle='--',
            errorbar = None,  estimator="mean", palette = 'Set2')

# Customizing the plot
plt.xlabel('')
plt.ylabel('Profit ($)')
plt.title("Average Monthly Profit Trends by Category (2016-2017)", fontsize=11)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.2)

# Show the plot
plt.legend(bbox_to_anchor=(1, 0.9))
plt.tight_layout()
plt.show()

<font size="3">
It is observed that:

- Over the last two years, furniture items have comparatively yielded lesser profits  ranging between &#36;-2500 and &#36;2800.
- Although technology is a high profit yielding category but is also the most volatile, where some months especially Oct 2016 and Mar 2017 had record profit of &#36;11,700 and &#36;11,000 respectively. Nov 2016 and Apr 2017 however, dented the superstore bearing losses of &#36;1500 and &#36;2600 respectively.
- Office Supplies profits have also been generally on the rise, with Dec 2016 bringing in highest profit of approx &#36;11,460.
- Over the last couple of months, the profits for technology and office supply items have dropped while that of furniture has risen.   


</font>

In [ ]:
# Profit distribution across Product Category

month_category.groupby('Category')['Profit'].describe()

In [ ]:
# Monthly Profit across Product Categories for 2016 and 2017

X = month_category.groupby(['Month-Year', 'Category'])['Profit'].mean()

In [ ]:
print(X)

In [ ]:
# Total Cost, Profit and Sales across Sub-categories of Office Supply items in Q4, 2017

month_subcategory = df[(df['Year'] == 2017) & (df['Quarter'] == 'Q4') & (df['Category'] == 'Office Supplies')].groupby(['Month', 'Sub-Category'])[['Profit','Sales','Cost']].sum().reset_index().sort_values('Month')

In [ ]:
# Adding Profit Margin column

month_subcategory['Profit Margin'] = (month_subcategory['Profit']/month_subcategory['Sales'])*100

In [ ]:
# Average Profit Margin by Sub-Category of Office Supply items for Q4 of 2017

#Barplot
fig, axes = plt.subplots(nrows=3,ncols =1 ,figsize=(8,15))

#first plot: profit margin

# plot graph
ax1 = sns.barplot(month_subcategory, x = 'Sub-Category', y = 'Profit Margin', estimator='mean',errorbar= None, ax=axes[0], palette = 'Set2')

# annotate data labels
for p in ax1.patches:
    ax1.annotate(
        format(p.get_height(), '.1f'),  # format height to 1 decimal
        (p.get_x() + p.get_width() / 2., p.get_height()),  # position at center of bar
        ha = 'center',  # horizontal alignment
        va = 'bottom',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(0, 1),  # small shift up
        textcoords='offset points'
    )

# Customize plot
axes[0].set_xlabel('')
axes[0].set_ylabel('Profit Margin (%)')
axes[0].set_title("Average Monthly Profit Margin Trends by Sub-Category (Q4 2017)", fontsize=11)
axes[0].tick_params(axis='x', labelrotation=45)

#second plot: sales
ax2 = sns.barplot(month_subcategory, x = 'Sub-Category', y = 'Sales', estimator='mean',errorbar= None, ax=axes[1],palette = 'Set2')

for p in ax2.patches:
    ax2.annotate(
        format(p.get_height(), '.1f'),  # format height to 1 decimal
        (p.get_x() + p.get_width() / 2., p.get_height()),  # position at center of bar
        ha = 'center',  # horizontal alignment
        va = 'bottom',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(0, 1),  # small shift up
        textcoords='offset points'
    )
axes[1].set_xlabel('')
axes[1].set_ylabel('Sales ($)')
axes[1].set_title("Average Monthly Sales Trends by Sub-Category (Q4 2017)", fontsize=11)
axes[1].tick_params(axis='x', labelrotation=45)

#third plot: profit
ax3 = sns.barplot(month_subcategory, x = 'Sub-Category', y = 'Profit', estimator='mean',errorbar= None, ax=axes[2],palette = 'Set2')

for p in ax3.patches:
    ax3.annotate(
        format(p.get_height(), '.1f'),  # format height to 1 decimal
        (p.get_x() + p.get_width() / 2., p.get_height()),  # position at center of bar
        ha = 'center',  # horizontal alignment
        va = 'bottom',  # vertical alignment
        fontsize=10,
        color='black',
        xytext=(0, 1),  # small shift up
        textcoords='offset points'
    )
axes[2].set_xlabel('')
axes[2].set_ylabel('Profit ($)')
axes[2].set_title("Average Monthly Profit Trends by Sub-Category (Q4 2017)", fontsize=11)
axes[2].tick_params(axis='x', labelrotation=45)

# Display plots
plt.tight_layout()
plt.show()

- The drop in Office Supplies profit margin in Dec 2017 is specifically due to Binders which despite second highest sales in Q4 has caused superstore to loose approx &#36;700.
- Considering Binders is low profit margin sub-category, the superstore needs to see whether it was caused by internal decisions such as heavy discounting to clear year-end inventory (in which case the low profit margin would have been anticipated) or externals factors influenced it such as Hurricane Harvey in Aug 2017 or costs spike by shipping carriers.

######**Category + Discount vs Profit**: Analyze how discounts affect profitability across categories

In [ ]:
# Adding Profit Margin column

df_wd['Profit Margin'] = (df_wd['Profit']/df_wd['Sales'])*100

In [ ]:
# Profit vs Weighted Discount Rate across Category

# plot graph
g = sns.lmplot(
    data= df_wd ,
    x='Weighted Discount Rate',
    y='Profit',
    col='Category',
    height=6,
    ci= None,
    palette = 'gist_stern',
    scatter_kws={'alpha': 0.6}
)

# customize plot
g.fig.suptitle("Profit by Weighted Discount Rate", fontsize=10)
g.fig.subplots_adjust(top=0.89)
g.set_axis_labels("Weighted Discount Rate", "Profit ($)")

# display plot
plt.show()

 - Technology is most sensitive to discounts, while furniture and office supplies appear less affected overall.
 - Technology category displays a noticeable negative trend in its regression line.  Higher discounts in this category appear to have a more direct and negative impact on profitability compared to furniture and office supplies because technology products have higher COGS hence thinner profit margins.
 - Technology has higher price elasticity of demand (a larger change in quantity demanded for a given change in price): hence although higher discounts trigger larger sale spikes but erodes the profit due to thin profit margins.
 - Due to rapid tech innovation and new models being released, the seller is compelled to offer higher discounts to prevent inventory write-offs on old models. Furniture items are bought primarily for convenience/quality over price hence last longer reducing pressure to discount for clearance.
 - Tech consumers are price-sensitive due to online price comparasions and delay purchase in favor of significant discounts which slash profits as time passes. The outliers at high discount portrays such bargain hunters.

**Recommendation:**
 - different discounting strategies is necessary for different product categories to optimize profitability.
 - for tech products:
  - Cap discounts at 10–15% and bundle with services (e.g., warranties).
  - Use time-bound promotions (e.g., "48-hour flash sale") to limit margin erosion.

- for Furniture and Office Supplies:
  - Leverage discounts to drive volume (high margins can absorb it).

######**Segment + Ship Mode vs Sales/Profit**: Understand how different segments use shipping methods and their associated performance.

In [ ]:
a = (per_transaction.merge(order_region_segment, on='Order ID')).drop(columns = ['Total_Sales_x', 'Total_Profit_x', 'Total_Cost','Profit Margin_x', 'Total_Sales_y','Total_Profit_y', 'Profit Margin_y'], axis=1)

In [ ]:
merged_data = a.merge(df.groupby('Order ID').agg(Sales=('Sales', 'sum'),Profit=('Profit', 'sum'),Cost = ('Cost', 'sum') ).reset_index(), on='Order ID')

In [ ]:
profit_by_segment_shipmode = merged_data.pivot_table(index='Segment', columns='Ship Mode', values='Profit', aggfunc = 'mean')

In [ ]:
# Mean Profit Per Order/Transaction across Segments and Ship mode

plt.figure(dpi=100)

sns.heatmap(profit_by_segment_shipmode, linewidth=0.6, annot= True, cmap = 'viridis',fmt=".1f")

plt.show()

- In home office segment, significantly large profit is earned by first class shipping and significantly low by standard shipping.
- In corporate segment, same day shipping yeilds significantly lower profits while other ship modes yield comparable profits.
- In consumer segment, second class and standard class shipping yields lowest mean profit  while same day shipping yields relatively higher profit.

######**Region + Ship Mode vs Profit**: Analyze if some shipping modes are more efficient/profitable in certain regions


In [ ]:
profit_by_region_shipmode = merged_data.pivot_table(index='Region', columns='Ship Mode', values='Profit', aggfunc = 'mean')

In [ ]:
# Frequency of orders using each ship mode type across regions

merged_data.pivot_table(index='Region', columns='Ship Mode', values='Order ID', aggfunc = 'count')

In [ ]:
# Mean Profit Per Order/Transaction across Regions and Ship mode

plt.figure(dpi=100)

sns.heatmap(profit_by_region_shipmode, linewidth=0.6, annot= True, cmap = 'viridis',fmt=".1f")

plt.show()

- In West region, same day shipping yields significantly high profit while standard shipping brings in lowest mean profit per transaction.
- In South region, same day shipping has caused significant loss, on average the superstore loses &#36;45 per order that is shipped same day. Second class shipping is highest profit yielding in the region.
- In East region, same day shipping earns the highest average profit across board i.e. approx &#36;108 per transaction while second class shipping yields lower profits.
- In Central region, the profits are generally on the lower side, with first class and same day shipping yielding lowest avg profits per order across the board.  

######**Sub-Category + Region vs Profit (or Profit Margin)** : Find which sub-categories are strong in each region


In [ ]:
# Sales, Profit and Cost across each Region and Product Sub-Category

subcategory_region = df.groupby(['Region','Sub-Category'])[['Profit','Sales','Cost']].sum().reset_index()

In [ ]:
# Adding profit margin column

subcategory_region['Profit Margin'] = (subcategory_region['Profit'] / subcategory_region['Sales']) * 100

In [ ]:
# Sort values by Profit Margin within each Region

subcategory_region = subcategory_region.sort_values(by = ['Region','Profit Margin'], ascending=[True, False], axis=0)

In [ ]:
#Top 3 performing sub-categories in each region

subcategory_region.groupby('Region').head(3).sort_values(by = ['Region','Profit Margin'], ascending=[True, False], axis=0)

- Labels, Paper, Envelopes are sub-categories with the highest profit margin across all regions.
- Envelopes is sub-category with the highest profit margin of 46% across board.
- In Central and South regions, labels have highest profit margin of approx 44%.
- In East region, the sub-category with largest profit margin is paper, approx 45%.
- In West region, envelopes have the largest profit margin of approx 46%.


######**Pairplot of Sales, Profit, Discount, and Quantity (Colored by Segment or Category)**


In [ ]:
filtered_data = df[['Sales', 'Profit', 'Discount', 'Quantity','Segment','Category']]

In [ ]:
#pairplot
plt.figure(figsize =(10,6))

sns.pairplot(filtered_data, hue = 'Category', corner=True, markers=["s", "D","o"], plot_kws={'s': 20, 'alpha':0.5}, diag_kind = 'kde' ) #DIAGNOL SHOW KDE PLOT

plt.show()

##  Correlation Analysis

In [ ]:
corr_matrix = df.corr(numeric_only = True)

In [ ]:
sns.heatmap(corr_matrix, annot = True)

- Profit and Sales are moderately correlated.
- Gross Sales is highly correlated to Cost with pearson correlation coefficient of 0.98. This is expected since higher cost generally reflect in the sale price.
- Rest of the numeric features have a weak correlation.

##  Conditional Analysis

######**What are the average sales per segment across different regions?**

In [ ]:
# Regional Per Order/Transaction Sales Distribution by Segment across Regions

pd.pivot_table(order_region_segment, values = 'Total_Sales', index = 'Segment', columns = 'Region', aggfunc = ['mean','median', 'sum'])

It is observed that:
- Highest sales come from Consumer segment across all regions.
- On average, least sales per order were observed in Central region for the Home Office Segment i.e. approx &#36;409 while highest average sales interestingly came from the same segment in South region (approx &#36;567) despite least number of orders i.e. 131 total orders. This high average basket value indicates demand of pricier items by Home Office customers in the South as opposed to Central region.

In [ ]:
# Purchase Frequency of each Product Sub-Category by Home Office segment in South and Central regions

df[(df['Segment']=='Home Office') & (df['Region'].isin(['South','Central']))].groupby(['Region','Sub-Category'])['Order ID'].count().sort_values(ascending=False)

#Binders are most ordered by Home Office segment in South region and Central Region.

In [ ]:
# Unique Order ID data

unique_orders = df.drop_duplicates(subset=["Order ID"], keep='first').reset_index(drop = True)

In [ ]:
# Order frequency of each ship mode used by Home Office segment in the South region

unique_orders[(unique_orders['Segment']=='Home Office') & (unique_orders['Region'] == 'South')]['Ship Mode'].value_counts()

 - In South region: binders, papers and furnishings are the most purchased sub-categories by Home office customers. Since this demographic also used standard shipping for most orders which is expected to be the least expensive option (negating the possibility of shipping fees raising AOV), the higher average basket value can be credited to purchase of furnishings due to it's higher comparative price.   

######**How does discount level affect profitability across different categories?**

In [ ]:
# Discount vs Profit Margin

# set canvas size
plt.figure(figsize=(6, 4))

# plot graph
hb = plt.hexbin(
    x=df_wd['Discount'],
    y=df_wd['Profit Margin'],
    gridsize=30,
    cmap='viridis',
    mincnt=1
)

# customize plot
plt.colorbar(hb, label='Count')
plt.xlabel('Discount')
plt.ylabel('Profit Margin (%)')
plt.title('Discount vs Profit Margin', fontsize=11)
plt.grid(True, linestyle='--', alpha=0.3)

# display plot
plt.tight_layout()
plt.show()

In [ ]:
# Discount vs Profit Margin aross Product Categories

categories = df_wd['Category'].unique()
n = len(categories)

#create subplots
fig, axes = plt.subplots(1, n, figsize=(5 * n, 4), sharey=True)


# plot graphs
for i, cat in enumerate(categories):
    ax = axes[i]
    subset = df_wd[df_wd['Category'] == cat]
    hb = ax.hexbin(
        subset['Discount'], subset['Profit Margin'],
        gridsize=30, cmap='viridis', mincnt=1
    )
    ax.set_title(cat, fontsize=11)
    ax.set_xlabel("Discount")
    if i == 0:
        ax.set_ylabel("Profit Margin (%)")
    fig.colorbar(hb, ax=ax, label='Count')

# customize plot
#mplcursors.cursor(hover=True)
fig.suptitle("Discount vs Profit Margin by Category", fontsize=12)

# display plot
plt.tight_layout()
plt.show()

**Guide to analyze hexbin plot**
- Dense areas (bright spots) = most common discount–profit margin combos.
- If high discounts cause negative margins, it's a red flag.
- Some categories might tolerate discounts better due to higher markups.
- Look for trends—does profit steadily drop as discount increases?
- Look for sub-categories where higher discounts severely reduce profitability.
- If you see very low or negative margins at high discounts, consider flagging them for discount policy review.
- Patterns may differ across categories, helping target pricing strategies.
- If you see clusters of high discount & low/negative margin, that’s a concern.

**Technology:**
- Even with moderate to high discounts, profit margins generally stay positive
which shows Technology category handles discounts better, maintaining reasonable margins even at discount levels of 40%-60%.

**Furniture:**
- As discounts increase, profit margins sharply decrease.
- High discount levels (≥40%) frequently lead to significant losses (margins as low as -200%).

**Office Supplies:**
- Profit margins remain positive at low discounts but drop drastically at higher discounts (≥ 70%).
- Most transactions cluster around 0–20% discounts, and high-discount instances correlate with large losses.
- Cluster of high discounted orders (~80%) resulting in heavy losses (-150% to 250% margin) calls for immediate action.

**Recommendations:**
- Review discount policy: For furniture and office supply categories, discount levels should be capped at no higher than 20-25% to prevent significant losses, while higher discounts of upto 45-50% can be offered on technology products.
- Product bundling or value-added offers can be offered for office supply items instead of flat discounts to drive demand while safeguarding profitability.
- Technology products generally have higher markup which provides cushion to margins at higher discounts, hence leverage discount to boost sales volume for tech products.


######**What percentage of high discount orders (>30%) result in negative profit?**


In [ ]:
# Adding Profit column

subset_profit_wd  = weighted_discounts.merge(df.groupby('Order ID')['Profit'].sum(), on = 'Order ID')

In [ ]:
# Filtering high-discount orders

df_high_discount = subset_profit_wd[subset_profit_wd['Weighted Discount Rate'] > 0.3].reset_index(drop=True)

In [ ]:
# Proportion of high-discount orders resulting in a loss or profit

loss = df_high_discount[df_high_discount['Profit'] < 0].shape[0]   #664 high discount orders (>30%) result in loss
profit = df_high_discount[df_high_discount['Profit'] > 0].shape[0] #51 high discount orders (>30%) result in profit

print(f"{(loss/df_high_discount.shape[0])*100:.1f}% of high discount orders (>30%) result in loss while only {(profit/df_high_discount.shape[0])*100:.1f}% result in profit.")

- This signifies the need to reconsider the discount policy specifically across categories discussed earlier.

######**Which sub-categories have the highest loss percentage and highest profit percentge compared to sales i.e. highest and lowest profit margin?**

In [ ]:
subcategory = df.groupby('Sub-Category')[['Profit', 'Sales', 'Cost']].sum().reset_index()

# Filter only sub-categories with negative total profit
loss_df = subcategory[subcategory ['Profit'] < 0]

# Calculate Loss Percentage
loss_df['Loss %'] = (-loss_df['Profit'] / loss_df['Sales']) * 100

# Sort by Loss Percentage descending
loss_df = loss_df.sort_values(by='Loss %', ascending=False)


In [ ]:
# Display result

loss_df.set_index('Sub-Category')

In [ ]:
# Filter only sub-categories with positive total profit
profit_df = subcategory[subcategory['Profit'] > 0]

# Calculate Profit Percentage
profit_df['Profit %'] = (profit_df['Profit'] / profit_df['Sales']) * 100

# Sort by Profit Percentage descending
profit_df = profit_df.sort_values(by='Profit %', ascending=False)

In [ ]:
# Display result

profit_df.set_index('Sub-Category').head(4)

- Tables, bookcases and supplies have the highest negative profit margin e.g, on average for every &#36;1 the store earns by selling tables, it loses &#36;0.086.
- Labels, Papers and Envelopes have the highest profit margin e.g. on average the store earns &#36;0.44 on top of every &#36;1 earned by selling labels.

######**What is the probability of receiving a profit/loss given a discount rate?**


In [ ]:
# Tag profit label

df['Profit_Label'] = df['Profit'].apply(lambda x: 'Profit' if x > 0 else 'Loss')

In [ ]:
# Creating dataframe for frequency of orders that made a profit/loss and total orders for each discount level

profit_loss_by_discount = df.groupby(['Discount', 'Profit_Label'])['Order ID'].count().unstack().fillna(0).merge(
    df.groupby('Discount')['Order ID'].count().rename('Total'), on = 'Discount')

In [ ]:
# Adding Profit/Loss Probability

profit_loss_by_discount['Profit Probability (%)'] = ((profit_loss_by_discount['Profit']/profit_loss_by_discount['Total'])*100).round(2)
profit_loss_by_discount['Loss Probability (%)'] = ((profit_loss_by_discount['Loss']/profit_loss_by_discount['Total'])*100).round(2)

In [ ]:
# Display result

profit_loss_by_discount

The probability of receiving a profit/loss given a discount rate is as follows:

 - The probability of making a loss increases with discount rate generally.
 - 40% discount is the threshold beyond which (as a whole across all product categories) the superstore makes no profit, however, this can be further drilled down to flag specfic categories and sub-categories to be able to tailor discount policy accordingly.


######**Among unprofitable orders, which categories or regions dominate?**

In [ ]:
# Filtering loss-making orders

region_loss = order_region_segment[order_region_segment['Total_Profit']<0]

In [ ]:
# Proportion of loss-making orders across Regions

region_loss['Region'].value_counts(normalize=True)*100

- The highest number of unprofitable orders come from Central region (40%) while least from the South region (13%).

######**What is the average delivery delay (Ship Date - Order Date), and does it impact customer segments or product types?**

In [ ]:
# Changing data type of Delay column

unique_orders['Delay']= unique_orders['Delay'].astype(str)

In [ ]:
# Order frequency across each state shipped via Standard Class which by number of days delayed

delay_orders = pd.pivot_table(unique_orders[unique_orders['Ship Mode'] == 'Standard Class'], values = 'Order ID', index = 'State', columns = 'Delay', aggfunc = ['count']).fillna(0)

In [ ]:
delay_orders['Sub_Total'] = delay_orders[[('count', '3 days'),
            ('count', '4 days'),
            ('count', '5 days'),
            ('count', '6 days'),
            ('count', '7 days')]].sum(axis=1)

In [ ]:
# Grabbing inner column label in multi-index column label

delay_orders.columns = [lvl1 for lvl0, lvl1 in delay_orders.columns]

In [ ]:
# Renaming for clarity

delay_orders.rename(columns={'': 'Total'}, inplace=True)

In [ ]:
# Proportion of orders delayed by 7 days across each state

for index, row in delay_orders.iterrows():
 print(index, f"{((row['7 days']/row['Total'])*100):.2f}%")

In [ ]:
# Preparing required data

# initializing lists
proportion = []
states = []

# looping through dataframe
for index, row in delay_orders.iterrows():
 states.append(index)
 proportion.append(row['7 days']/row['Total'])

In [ ]:
# top 10 states by proportion of standard delivery orders delayed by 7 days

pd.Series(data = proportion,index = states, name = "Proportion of Orders Delivered in 7 Days").sort_values(ascending=False).head(10)

- Across all orders that have been shipped, on average it takes nearly 4 days for delivery.
- Standard class is the most widely used shipping mode across regions hence it is studied to narrow states where it is least efficient:
  - The store needs to focus on Nevada (33%), Rhode Island (25%), Arkansas (20%) and Alabama (19%) to eliminate logistical inefficiencies due to highest proportion of standard delivery orders being delayed by 7 days in these states to minimise customer dissatisfaction and increase customer lifecycle.

######**Are there any sub-categories that are profitable despite high discounting?**

In [ ]:
# Import library
from matplotlib.patches import Patch

hue_order = ['Technology', 'Office Supplies', 'Furniture']    # manually setting hue_order
palette = {                                                   # defining custom palette
    'Technology': 'green',
    'Office Supplies': 'blue',
    'Furniture': 'orange'
}

# Create FacetGrid object, 'g'
g = sns.FacetGrid(df_wd, col='Sub-Category', hue="Category",col_wrap=4,hue_order=hue_order, palette = palette)

# Mapping a plot to each facet
g.map_dataframe(sns.scatterplot, "Weighted Discount Rate", "Profit", alpha=0.6)

# Customizing plot
g.set_axis_labels("Weighted Discount Rate","Profit ($)")
g.set_titles(col_template="{col_name}")

# Adding reference line
g.refline(y=0, linestyle="-", color="gray", linewidth=0.8)

# Manually create legend with color markers
legend_elements = [Patch(facecolor=palette[cat], label=cat) for cat in hue_order]
g.fig.legend(handles=legend_elements, title='Category', loc='center right', bbox_to_anchor=(1.13, 0.5))

#g.add_legend()  # Add legend
#g._legend.set_bbox_to_anchor((1.13, 0.5))  # Move outside to the right (x, y)
#g._legend.set_title('Category')

# Show plot
g.tight_layout()
sns.set_style("darkgrid")
plt.show()

- Copiers are generally profitable despite being discounted up to 40%.
- Certain Technology products like machines, certain office supply products such as binders and certain furniture products such as bookcases and tables are non-profitable at higher discounts.
- Rest of SKUs are generally stable in their profits across low and high discount levels.

##  Business Insights and Recommendations

####**Executive Summary**

An analysis of 9,994 sales records from 2014 to 2017 for the Retail Superstore chain reveals an overall profit growth of approximately $25,000. However, a notable decline in profitability was observed in Q4 of 2017, particularly within the Technology and Office Supplies categories.

Among product segments, Copiers generated the highest average sales value, approximately $1,100 per order. In contrast, Tables, Bookcases, and Supplies incurred the highest per-order losses. Furthermore, discounting patterns show a strong correlation with profitability: 92.9% of orders with discounts exceeding 30% result in a loss, and the store experiences no profit at discount levels beyond 40%, across all product categories.

To support sustained profitability and strategic growth, the company should consider:

- Implementing data-driven, category-specific discounting strategies;

- Expanding efforts in the Central region, which shows potential for targeted growth;

- Conducting a root cause analysis to address consistently low profit margins within the Standard shipping class, with the aim of optimizing logistics and pricing decisions.

####**Insights Deep-Dive**

##### **Seasonal Trend**

Profitability:
- **Overall Growth**: Profit shows a general increasing trend from 2014 to 2017, with significant spikes in late 2016 and late 2017.
- **Seasonality**: Q4 of each year tends to have a strong spike in profits, suggesting seasonal effects (possibly due to holiday sales).
- **Recent Stability**: In 2017 Q3 and Q4, profits stabilize at a relatively high level compared to earlier years.
- **Product Level Performance**:
 - Over the last two years, furniture items have comparatively yielded lesser profits ranging between -&#36;2500 and &#36;2800.
 - Although technology is a high profit yielding category but is also the most volatile, where some months especially Oct 2016 and Mar 2017 had record profit of &#36;11,700 and &#36;11,000 respectively. Nov 2016 and Apr 2017 however, dented the superstore bearing losses of &#36;1500 and &#36;2600 respectively.
 - Office Supplies profits have also been generally on the rise, with Dec 2016 bringing in highest profit of approx &#36;11,460.
 - More recently over the last couple of months, the profits for technology and office supply items have dropped while that of furniture has risen.
 - Office supply items recent drop in profit is due to Binders which despite second highest sales in Q4 has caused superstore to loose approx $700.

Sales:
- **Steady Growth**: Sales show a strong and relatively consistent upward trend over the years.
- **Sharp Drops**: Noticeable sales drops occur in Q1 of each year.
- **Acceleration**: The sales growth rate accelerated particularly in 2017.

Profit Margin:
- **Initial Stability, Later Volatility**: Profit margins are fairly stable around 12–14% from 2014 to early 2016 but become much more volatile afterward.
- **High Peaks**: A major peak occurs in 2017 Q1 (around 19%), suggesting exceptional efficiency/profitability for that quarter.
-**Decoupling**: Sales and profits are growing, but margins are shrinking, indicating rising costs or inefficiencies.
- **Recent Decline**:
 - After peaking, profit margins decreased sharply towards the end of 2017 despite high sales.
 - The steepest drop in average profit margin  in the last quarter of 2017 is observed in Consumer segment in Central region (-3% to -17.6%) due to high discounts (60%-80%) given to it, and Home Office segment in South region (19% to 3%).
 - In South region, the discounts given to Home Office segment have ranged between 0-20% with only a couple of orders that have been discounted more than 50%. This indicates that the profit margin has been impacted by other factors such as costs/sale of low margin products there.



##### **Overall Product Performance**

- Technology category as a whole is the highest selling and earns highest profit on per order basis with average order value of approximately &#36;219 and profit of nearly &#36;31.

- Copier (sub-category bringing in most sales on average of &#36;1,100 per order) is being most ordered by consumers.

- Despite least average sales per order, Office Supplies items have the the highest profit margin per order on average of roughly &#36;31.

- Furniture has lowest average profit margin of &#36;10 despite roughly equal average sales as technology items, per order.

##### **Poduct Portfolio Analysis**

- Binders, papers and furnishings are the most purchased sub-categories by Home office customers in the South region, which is the demographic bringing in highest average sales per order (AOV).

- Since this demographic also used standard shipping for most orders which is expected to be the least expensive option (negating the possibility of shipping fees raising AOV), the higher average basket value can be credited to purchase of furnishings due to it's higher comparative price.

- Tables, bookcases and supplies have the highest negative profit margin e.g, on average for every &#36;1 the store earns by selling tables, it loses &#36;0.086, while labels, papers and envelopes have the highest profit margin e.g. on average the store earns &#36;0.44 on top of every $1 earned by selling labels.

- Copiers are generally profitable despite being discounted up to 40%.
Specific products including machines (tech), binders (office supply), bookcases and tables (furniture) are non-profitable at higher discounts.

##### **Regional Analysis**

- Largest order volume originates from the West region (32%) closely followed by East region (28%).

- West region outperforms others with approximately 15% more profit generated compared to East region which is the next most profitable region on per order basis.

- Central Region is the most under-performing region with cumulative profit of approx $40,000 which is roughly 3x lesser than West region, and yields the highest number of unprofitable orders (40%).

- In the East, South and West regions, consumer segment brings in the biggest share of profit per transaction while corporate segment is biggest profit contributor in Central region (despite consumer segment bringing in the highest order volume).

- Unlike other regions where median profit margin (per order basis) is stable across segments, in Central region: the profit margin for Consumer segment (&#36;9.5) is nearly half of Home Office segment (&#36;19).

- On average, least sales per order are observed in Central region for the Home Office Segment i.e. approx (&#36;409) while highest average sales interestingly come from the same segment in South region (approx $567) despite least number of orders i.e. 131 total orders.

##### **Customer Segmentation**

- Consumer segment brings in the largest order volume across all regions and also the most profit ($ 0.13 million to date which is twice that of the Home Office segment).

- On per order basis, highest median profit comes from corporate orders ($16.8) which is expected since they either buy bulk quantity or items higher on the price band.

- Besides standard class, all ship modes yield high average profits per order in the Home office segment (excess of &#36;85). First class shipping yields &#36;93 per order on average which is the highest across board.

- In Corporate segment, same day shipping yields significantly lower profits per order on average (&#36;31, lowest across board) compared to other shipping modes that yield comparable profits of roughly &#36;60.

- Consumer segment yields comparative profits per order on average through different ship modes ranging between &#36;50 (second-class shipping) to &#36;63 (same day shiping).

##### **Discount Effectiveness**

- The probability of making a loss increases with discount rate generally.
40% discount is the threshold beyond which (as a whole across all product categories) the superstore makes no profit.

- Overall, 92.9% of high discount orders (>30%) result in loss while only 7.1% result in profit.

- Technology is most sensitive to discounts, while furniture and office supplies appear less affected overall.

- Technology category displays a noticeable negative trend where higher discounts appear to have a more direct and negative impact on profitability compared to furniture and office supplies.

- For furniture items, high discount levels (≥40%) frequently lead to significant losses (margins as low as -200%).

- For office Supply items, profit margins remain positive at low discounts but drop drastically at higher discounts (≥ 70%) resulting in heavy losses (-150% to -250% margin) which calls for immediate action.

##### **Shipping Mode and Delays Trend**

- Across all orders that have been shipped, on average it takes nearly 4 days for delivery.

- Standard Class yields the lowest profit margin despite highest total sales and profit (since it is most widely used). This could be because it is cost-intensive (eating into profits due to high logistics costs from shipping partners).

- Despite similar AOV and median profit per transaction/order, Standard class has lowest profit margin (&#36;12.1) while First class has highest profit margin (&#36;13.9) per transaction/order basis.

- Nevada (33%), Rhode Island (25%), Arkansas (20%) and Alabama (19%) are most susceptible to order delays evident by highest proportion of standard delivery orders being delayed by 7 days in these states.

- Central region has been struggling in making profit compared to other regions amongst all shipping modes with with first class and same day shipping yielding lowest average profits per order across the board, &#36;21 and &#36;25 respectively.

- Same day shipping is out-performing in East and West bringing in roughly &#36;108 and &#36;91 per order, respectively, however, in South it has caused significant loss, on average the superstore loses $45 per order that is shipped same day.


####**Recommendations**

##### **Product Startegy**

- Plan on raising inventory levels of office supplies items but re-consider pricing or discount of furniture items (specifically low profit earners at higher discounts such as bookcases and tables).
- Specifically, focus on efficent inventory forecasting for high-sale items such as Copiers and consider phasing out Machines from the product portfolio due to significant losses in range of &#36;2000  to &#36;4000.
- Although technology profits are volatile, the store can rely on phone sales for being majorly profitable.
- Focus marketing efforts on SKUs with high profit-margins across all regions such as labels, paper, and envelopes.
- Re-evaluate discount policy or re-negotiate costs with manufacturers/logistic partners for low profit-margin SKUs including tables, bookcases and supplies.

##### **Regional Customer Growth and Retention**

- Target Consumer segment (specially in the Central region) with product bundling offers of high-margin products or upsell incentives to boost average order basket value such as free shipping for minimum spend in specific high margin product categories.

- Ensure the longevity of Corporate segment's high profitability in the Central region by offering incentized offerings such subscription-like contracts or flat discounts on bulk purchases of high-margin products to safeguard profitability.

-  Replicate effective strategies of Home Office segment from the South region in Central region to boost it's average order value including product pricing, discount thresholds and limited time incentives.

- Leverage recommendation engines to offer personalised product offerings and advertisments based on purchase patterns towards Consumer segment specially in the West and East regions which bring in the highest order volume and overall profit.

##### **Discount Optimization**

- Implementation of tailored discounting strategies are necessary for different product categories to optimize profitability:
 - For Technology products:
   - Higher markup provides cushion to margins at higher discounts, hence leverage discount (up to  45-50%) to boost sales volume.
   - Use time-bound promotions (e.g., "48-hour flash sale") and bundle with services (e.g., warranties) to limit margin erosion.
 - For Furniture and Office Supplies:
   - Cap discount levels at no higher than 20-25% to prevent significant losses.
   - Offer product bundling or value-added offers for office supply items instead of flat discounts to drive demand while safeguarding profitability.


##### **Addressing Logistical Shortcomings**

- Cut down logistical inefficiencies for same day shipping in the South region by adopting best practises from the West and the East such as warehouse processes and inventory allignment with the demand tracked most preferably in real-time.

- Investigate the driver of low profit margin for Standard class shipping:
 - Re-negotiate bulk rate discounts for standard class carriers to improve the profit margin if it is caused by higher fixed logistical costs.
 - Consider bundling offers with high-margin products against incentives such as limited discounts or shipping fee cuts if majorly low-margin product SKUs are being purchased through standard shipping mode.
 - If driven by high promotional discounts, shift toward loyalty-based or reward-point based incentives instead of flat discounts.

- Central region calls for root cause identification:
  -  Identify cost inefficiencies (e.g., last-mile delivery, warehouse distance, low route density) and benchmark these costs against other regions.

 - Reassess Product-Shipping Fit: if low-margin products are disproportionately being shipped via First Class or Same Day shipping then restricting them for such SKUs is recommended.


####**Assumptions, Caveats and Next Steps**

**Discount Application Assumption**: It is assumed that the discount value in each transaction record is applied to the total quantity purchased, not just to a single unit.

**Cost Data Limitation**: The absence of cost-related data—specifically unit cost price (COGS) and logistics costs—limits the ability to fully explain certain observations. For example, the low profit margins associated with the Standard shipping class cannot be conclusively attributed to discounting without ruling out cost as a potential driver. Similarly, identifying low-margin SKUs is constrained by this data gap.

**Next Steps and Recommendations**: To validate or refute potential business loss drivers, such as the hypothesis that aggressive discounting on items primarily shipped via Standard class is reducing profit margins, the store should consider conducting further statistical analyses. This includes A/B testing or controlled experiments to determine the true impact of discount strategies on profitability.